In [261]:
from bs4 import BeautifulSoup
import requests
import time
from urllib.request import urlopen, Request

In [262]:
import urllib.request

headers = {'User-Agent': 'Mozilla/5.0'}

book_titles = []
book_descriptions = []
book_genres = []

page_link = "https://www.goodreads.com/list/show/4093.Best_Books_of_the_Decade_2010s?page=1"
page = requests.get(page_link, headers=headers)
soup = BeautifulSoup(page.content, 'html.parser')

for i in range(50):
    idx = 0
    print(page_link)
    books = soup.find_all('a', class_='bookTitle')
    
    for i in books:
        link = "https://goodreads.com" + i['href']
        time.sleep(1)
        resp = requests.get(link, headers=headers)

        book_soup = BeautifulSoup(resp.content, 'html.parser')
        book_descriptions.append(book_soup.find("meta", property="og:description").get('content'))
        book_titles.append(book_soup.find("meta", property="og:title").get('content'))
        genre_buttons = []
    
        genre_buttons = book_soup.find_all(class_="actionLinkLite bookPageGenreLink")
        if(len(genre_buttons) == 0):
            genre_buttons = book_soup.find_all(class_='BookPageMetadataSection__genreButton')
    
        single_book_genres = []
        for j in genre_buttons:
            single_book_genres.append(j.text)
        print(single_book_genres)
        book_genres.append(single_book_genres)
    
        idx=idx+1
        if idx == 5:
            break
    page_link = 'https://goodreads.com' + soup.find('a', class_='next_page').get('href')
    page = requests.get(page_link, headers=headers)
    soup = BeautifulSoup(page.content, 'html.parser')
    


https://www.goodreads.com/list/show/4093.Best_Books_of_the_Decade_2010s?page=1
['Young Adult', 'Romance', 'Fiction', 'Contemporary', 'Realistic Fiction', 'Teen', 'Coming Of Age']
['Young Adult', 'Dystopia', 'Fiction', 'Fantasy', 'Science Fiction', 'Romance', 'Adventure']
['Young Adult', 'Science Fiction', 'Dystopia', 'Fiction', 'Fantasy', 'Science Fiction', 'Romance', 'Adventure', 'Young Adult', 'Teen', 'Apocalyptic', 'Post Apocalyptic', 'Audiobook']
['Fiction', 'Mystery', 'Thriller', 'Thriller', 'Mystery Thriller', 'Mystery', 'Crime', 'Suspense', 'Contemporary', 'Adult', 'Audiobook', 'Adult Fiction']


KeyboardInterrupt: 